In [1]:
from langgraph.graph import StateGraph, END, add_messages
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_ollama import ChatOllama
from langchain_tavily import TavilySearch
from langchain_core.messages import BaseMessage, AIMessage
from langgraph.prebuilt import ToolNode
from typing import TypedDict, Annotated, Sequence
from dotenv import load_dotenv
import os


load_dotenv()
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
# model = ChatOllama(model="llama3.2")
search_tool = TavilySearch(max_results=2, search_depth = 'basic')

tools = [search_tool]
tool_model = model.bind_tools(tools=tools)

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

def chat_node(state: AgentState) -> AgentState:
    response = tool_model.invoke(state['messages'])
    return {"messages": [response]}

def tool_router(state: AgentState):
    last_msg = state['messages'][-1]
    if isinstance(last_msg, AIMessage) and getattr(last_msg, "tool_calls", None):
        return "tool_node"
    else:
        return END
    
tool_node = ToolNode(tools=tools)

graph = StateGraph(AgentState)

graph.add_node("chat_node", chat_node)
graph.add_node("tool_node", tool_node)

graph.set_entry_point("chat_node")
graph.add_conditional_edges("chat_node", tool_router)
graph.add_edge("tool_node", "chat_node")
app = graph.compile()

Stream for SSE(Server Sent Events) or one token at a time with .astream_events

In [2]:
from langchain_core.messages import HumanMessage
initial_state = {"messages": [HumanMessage(content="what is the weather today in Kolkata, India")]}

events = app.astream_events(input=initial_state, version="v2")

async for event in events:
    print(event)

{'event': 'on_chain_start', 'data': {'input': {'messages': [HumanMessage(content='what is the weather today in Kolkata, India', additional_kwargs={}, response_metadata={})]}}, 'name': 'LangGraph', 'tags': [], 'run_id': '494a2b3c-d39d-4f81-9182-853b009c7c19', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {'input': {'messages': [HumanMessage(content='what is the weather today in Kolkata, India', additional_kwargs={}, response_metadata={}, id='f7c10ce6-ffaa-4452-b816-1cd8aed2bc69')]}}, 'name': 'chat_node', 'tags': ['graph:step:1'], 'run_id': '6c3c968a-6a15-4b5b-b9e0-21f5f24b947e', 'metadata': {'langgraph_step': 1, 'langgraph_node': 'chat_node', 'langgraph_triggers': ('branch:to:chat_node',), 'langgraph_path': ('__pregel_pull', 'chat_node'), 'langgraph_checkpoint_ns': 'chat_node:fe223c69-1c5d-8fe0-1d09-21a16f97715f'}, 'parent_ids': ['494a2b3c-d39d-4f81-9182-853b009c7c19']}
{'event': 'on_chat_model_start', 'data': {'input': {'messages': [[HumanMessage(content='what i

How to refer to each token in nodes?
- stream the outputs of on_chat_model_stream
- data is event['data']['chunk'].content 

In [15]:
initial_state = {"messages": [HumanMessage(content="what is the weather today in Kolkata, India")]}

events = app.astream_events(input=initial_state, version="v2")

async for event in events:
    if event.get("event") == "on_chat_model_stream":
        content = event['data']["chunk"].content
        print(content, end='', flush=True)

[{'type': 'text', 'text': 'The weather in Kolkata, India on November 8, 2025, is expected to be around 30.2°C with mist. The wind speed will be 16.6 kph from the NNW,', 'extras': {'signature': 'CiIB0e2Kbxh/SgyoQLtJVd65MpbvJnJVvI3sElXEb/78omvwCmkB0e2Kb4kHhtQTX7ZmGOSFUBmdhnUi2klhwsR/B6s1BvSuaH6gqQwb9rY/0M6UuRDqVIU3WqaQc79i7CNIrJs8vzr5U5TZVqtGaWgP8ECez3iZv1MasisW90L557BZHj7uck3OsypHGaoK7wEB0e2Kb4kWLLgMLFqNvDEyofZ5n+QSNAU8Mk55JJfuBHAlmMR/JmqBVMPHSiJfYVTzGj4X3tWttuSPKqfplczH5NLnhuxSng7kdQgtiTtgovLgub/mgdNALatRdVBF8xljrSaUH2BMmj3D99EB7m/39YPEy9T4W+oYACeKMRtkVOA42CcdHahbmD4uy1fab1NIYT9WoqQvOkwQYI4fk19F9PmRmGEiU6lWNgQ5yBX6fuS1wGUEp8/Wt5LG8pU3AgreMv7gxU6rt8c7f9IFxx0e8gEhXGpp3DMH3Z6dbdySfI/FVJs1p818X1h95PILKQrJAQHR7YpvHG2fdkEhBhrE1ZDz2B6uaUafC5HhIdV9os1VIkzyCsBUnQ8WHiDGN+bL9+INhF1z3adbYexAL5kisjE5ImfX5tRw6xturjtNao4n1BkWMb5E7EeUHgUu3/GmvzdEL87bJkWldsao/o+n3iqcc0y29uT6HhdP+FoGIFIl8JfpGQA7H+2WlYWYsXQ3ybJqJRizyKG7g8rGhAWI42QAr89fKGECiP7+7e1+ABq2JVRG8lWkdfd2kfnDnBX4MrdUCanM0OxAuQrdAQHR7Ypvh7skAKFQl0